In [181]:
# Load github repo
!git clone https://github.com/JBlank19/SongClassifier
repo_path = "SongClassifier"

fatal: destination path 'SongClassifier' already exists and is not an empty directory.


In [182]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [183]:
# Load the dataset
dataset = pd.read_parquet("SongClassifier/ultraclean_genre_dataset.parquet")

# Display the first few rows of the dataset
dataset.head()


,name,artists,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year,genres
0,Testify,['Rage Against The Machine'],False,0.470,0.978,7,-5.399,1,0.0727,0.02610,0.000011,0.3560,0.503,117.906,210133,4.0,1999,"[""alternative metal"", ""alternative rock"", ""con..."
1,Guerrilla Radio,['Rage Against The Machine'],True,0.599,0.957,11,-5.764,1,0.1880,0.01290,0.000071,0.1550,0.489,103.680,206200,4.0,1999,"[""alternative metal"", ""alternative rock"", ""con..."
2,Calm Like a Bomb,['Rage Against The Machine'],False,0.315,0.970,7,-5.424,1,0.4830,0.02340,0.000002,0.1220,0.370,149.749,298893,4.0,1999,"[""alternative metal"", ""alternative rock"", ""con..."
3,Mic Check,['Rage Against The Machine'],True,0.440,0.967,11,-5.830,0,0.2370,0.16300,0.000004,0.1210,0.574,96.752,213640,4.0,1999,"[""alternative metal"", ""alternative rock"", ""con..."
4,Sleep Now In the Fire,['Rage Against The Machine'],False,0.426,0.929,2,-6.729,1,0.0701,0.00162,0.105000,0.0789,0.539,127.059,205600,4.0,1999,"[""alternative metal"", ""alternative rock"", ""con..."


In [184]:
###### GET ALL GENRES LIST IN DATASET ######

import pandas as pd
import ast  # for safely evaluating strings as Python literals

# Load the dataset
# dataset = pd.read_parquet("SongClassifier/ultraclean_genre_dataset.parquet")

def safe_literal_eval(val):
    try:
        return ast.literal_eval(val)
    except ValueError:
        return []  # or some other default like ['Unknown']

# Convert the string representation of lists back to lists
dataset['genres'] = dataset['genres'].apply(safe_literal_eval)

# Extract and flatten the list of genres
all_genres = [genre for sublist in dataset['genres'] for genre in sublist]

# Get distinct genres
distinct_genres = set(all_genres)

# Convert the set back to a list if you need it as a list
distinct_genres_list = list(distinct_genres)

# Print or inspect the distinct genres
print(distinct_genres_list)
print(len(distinct_genres_list))


['finnish electro', 'kawaii metal', 'egyptian traditional', 'danish pop', 'technical grindcore', 'cumbia boliviana', 'portsmouth indie', 'hungarian classical performance', 'japanese melodic punk', 'canadian stoner rock', 'karneval', 'brega romantico', 'new jersey rap', 'australian psych', 'malian blues', 'persian hip hop', 'portuguese experimental', 'indie jazz', 'hamilton on indie', 'belgian post-rock', 'myanmar indie', 'classic uk pop', "black 'n' roll", 'deathgrass', 'burmese pop', 'serialism', 'dub poetry', 'nashville sound', 'japanoise', 'tajik traditional', 'indie pop rap', "jackin' house", 'chanson', 'trap mexicano', 'screamo', 'classical mezzo-soprano', 'greek hip hop', 'cascadian black metal', 'austrian orchestra', 'danish black metal', 'bagpipe', 'deep smooth r&b', 'milwaukee indie', 'flick hop', 'cantopop', 'rogaland musikk', 'latin hip hop', 'cape town indie', 'experimental indie', 'deep tech house', 'post-romantic era', 'maine indie', 'slack-key guitar', 'canadian celtic',

In [185]:
def reclassify_genre(genre_list):
    super_genres = []
    unclassified_genres = []

    # Define keywords for each super genre
    genre_keywords = {
    'Industrial/Gothic': ['industrial', 'gothic', 'dark wave', 'neue deutsche härte', 'ethereal wave'],
    'Metal': ['metal', 'grindcore', "black 'n' roll", 'ukhc', 'deathcore', 'djent', 'black metal', 'doom metal'],
    'Rock/Indie': ['rock', 'indie', 'punk', 'guitar', 'screamo', 'post-romantic', 'hard rock', 'garage rock', 'psychedelic rock', 'folk rock', 'alternative rock', 'post-rock', 'shoegaze'],
    'Pop': ['pop', 'synthpop', 'electropop', 'pop rock', 'pop punk', 'k-pop', 'j-pop'],
    'Country': ['country', 'deathgrass', 'nashville sound', 'americana', 'bluegrass', 'country rock', 'outlaw country', 'tex-mex'],
    'Blues/Jazz': ['blues', 'jazz', 'r&b', 'rhythm and blues', 'soul', 'funk', 'swing', 'bebop', 'hard bop', 'cool jazz', 'jazz fusion'],
    'Reggae': ['reggae', 'ska', 'dancehall', 'dub', 'ragga'],
    'Rap/Hiphop': ['rap', 'hiphop', 'hip-hop', 'hip hop', 'hop', 'trap', 'gangsta rap', 'east coast hip hop', 'west coast hip hop', 'southern hip hop'],
    'EDM': ['edm', 'electronic', 'dance music', 'electro', 'house', 'techno', 'trance', 'drum and bass', 'dubstep', 'big room', 'deep house', 'progressive house', 'tech house', 'electro house'],
    'Urban/Reggaeton': ['urban', 'reggaeton', 'latin trap', 'bachata', 'merengue'],
    'Classical': ['classical', 'orchestra', 'baroque', 'romantic era', 'serialism', 'chamber music', 'opera', 'symphonic'],
    'Folk/Traditional': ['folk', 'traditional', 'tradicional', 'brega', 'copla', 'bagpipe', 'celtic', 'chanson', 'world music', 'ethnic', 'native american', 'andalusian classical', 'medieval'],
    'Miscellaneous': []
}

    # Check each genre in the list against the keywords
    for genre in genre_list:
        classified = False
        for super_genre, keywords in genre_keywords.items():
            if any(keyword in genre.lower() for keyword in keywords):
                super_genres.append(super_genre)
                classified = True
                break
        if not classified:
            super_genres.append('Miscellaneous')

    # Combine super genres and unclassified genres
    return super_genres

# Apply the function to the dataset
dataset['super_genres'] = dataset['genres'].apply(reclassify_genre)

# Optionally, display the updated dataset
dataset.sample(n=5)


,name,artists,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year,genres,super_genres
746180,porcelain,['mxmtoon'],False,0.569,0.108,9,-15.855,1,0.0350,0.88300,0.000000,0.1120,0.1810,110.421,271915,4.0,2018,"[alt z, bedroom pop, electropop, indie pop, pop]","[Miscellaneous, Pop, Pop, Rock/Indie, Pop]"
496834,Samba,"['Mansour Seck', 'Ousmane Hamady Diop']",False,0.572,0.564,7,-13.254,1,0.0285,0.75000,0.000265,0.1170,0.8160,102.367,290933,3.0,1995,"[desert blues, griot, mbalax, musique peule, s...","[Blues/Jazz, Miscellaneous, Miscellaneous, Mis..."
825545,Spain (feat. Tory Lanez and Jackboy),"['Kodak Black', 'Jackboy', 'Tory Lanez']",True,0.882,0.644,5,-7.721,1,0.1170,0.03060,0.000000,0.0956,0.1550,133.031,285531,4.0,2020,"[florida rap, hip hop, melodic rap, miami hip ...","[Rap/Hiphop, Rap/Hiphop, Rap/Hiphop, Rap/Hipho..."
360792,"Aida: Act II: Scene 1: Vieni, sul crin ti piovano","['Giuseppe Verdi', 'Erich Leinsdorf', 'London ...",False,0.361,0.211,0,-18.595,1,0.0462,0.98400,0.000119,0.6430,0.0733,77.220,103360,5.0,1971,"[classical, italian opera, italian romanticism...","[Classical, Classical, Miscellaneous, Classica..."
10764,Real Love,['H-Blockx'],False,0.495,0.972,9,-6.722,1,0.0818,0.00524,0.000000,0.1540,0.3980,109.318,207373,4.0,1994,"[german alternative rock, german rock]","[Rock/Indie, Rock/Indie]"


In [186]:
# Extracting distinct genres from the 'genre' column
distinct_genres = set()
for genres in dataset['super_genres']:
    distinct_genres.update(genres)

distinct_genres_list = list(distinct_genres)
#distinct_genres_list.sort()

print(len(distinct_genres_list))
print(distinct_genres_list)

13
['Blues/Jazz', 'Urban/Reggaeton', 'Folk/Traditional', 'Rock/Indie', 'Rap/Hiphop', 'Miscellaneous', 'Classical', 'EDM', 'Industrial/Gothic', 'Reggae', 'Metal', 'Country', 'Pop']


In [187]:
# Mapping of super genres to their assigned numbers
genre_number_mapping = {
    'Industrial/Gothic': 0,
    'EDM': 1,
    'Reggae': 2,
    'Metal': 3,
    'Blues/Jazz': 4,
    'Country': 5,
    'Urban/Reggaeton': 6,
    'Rap/Hiphop': 7,
    'Rock/Indie': 8,
    'Pop': 9,
    'Classical': 10,
    'Folk/Traditional': 11,
    'Miscellaneous': 12
}

# Assuming df is your DataFrame and 'super_genres' is the column with genres
def assign_genre_number(row):
    genres = row['super_genres']  # Adjust splitting logic based on your data format
    if len(genres) == 1:
        return genre_number_mapping[genres[0]]
    else:
        # Using the main genre method
        return min(genre_number_mapping[genre] for genre in genres)

dataset['super_genre'] = dataset.apply(assign_genre_number, axis=1)

dataset.drop(columns=['genres', 'super_genres'])

dataset.sample(n=5)


,name,artists,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year,genres,super_genres,super_genre
700021,Until You Come Back to Me (That's What I'm Gon...,['Aretha Franklin'],False,0.664,0.6340,2,-6.348,1,0.0349,0.68300,0.000033,0.2210,0.6660,93.914,208467,4.0,1974,"[classic soul, jazz blues, memphis soul, soul,...","[Blues/Jazz, Blues/Jazz, Blues/Jazz, Blues/Jaz...",4
568698,"Sonata No. 9 in G Minor, RV 42: III. Sarabande...","['Antonio Vivaldi', 'Anner Bylsma', 'Hidemi Su...",False,0.179,0.0686,6,-20.649,0,0.0442,0.86600,0.625000,0.1070,0.0565,167.675,170907,3.0,2002,"[baroque, classical, early music, italian baro...","[Classical, Classical, Miscellaneous, Classica...",10
761342,Kiss the Clown,"[""Enuff Z'Nuff""]",False,0.457,0.9200,9,-8.614,1,0.0551,0.00394,0.004050,0.1110,0.5590,141.334,195773,4.0,2019,"[glam metal, hard rock, sleaze rock, undergrou...","[Metal, Rock/Indie, Rock/Indie, Pop]",3
692280,Forever Laughing - Demo,['The Beloved'],False,0.466,0.9460,8,-4.297,1,0.0360,0.06000,0.676000,0.2830,0.8980,94.922,276842,4.0,2020,"[hip house, new romantic, new wave, new wave p...","[EDM, Miscellaneous, Miscellaneous, Pop, Pop]",1
61268,Talkin' New York,['Bob Dylan'],False,0.520,0.4430,7,-7.755,1,0.0399,0.81100,0.001040,0.0489,0.7660,123.900,197653,4.0,1962,"[album rock, classic rock, country rock, folk,...","[Rock/Indie, Rock/Indie, Rock/Indie, Folk/Trad...",8


In [189]:
dataset.to_parquet('cleaned_super_genre_dataset.parquet')